# Θ-Mechanism Extraction from Optical Conductivity
## Complete Pipeline: σ(ω,T) → M(ω) → E,S → Θ → Θ_c/T_c

**Theory:** Adaptonic Information Temperature Mechanism  
**Material:** LSCO x=0.15 (Standard cuprate)  
**Expected:** Θ_c/T_c = 1.45 ± 0.12  

---

### Pipeline Overview:
```
Raw Data → Memory Function → Spectral Measure → Information Temperature → Critical Point
σ(ω,T)   → M(ω) = f[σ]    → π(ω) = σ/∫σ      → Θ = ∂E/∂S           → Θ_c at T_c
```

### Falsification Gates:
1. **KK Consistency:** M(ω) must satisfy Kramers-Kronig
2. **Method Agreement:** Θ_c from 3 methods must agree within 10%
3. **Classification:** Θ_c/T_c must match structural class (standard vs infinite-layer)
4. **ε(ω) Sensitivity:** Results robust to energy kernel choice


## 0. Setup & Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate, interpolate, optimize
from scipy.signal import savgol_filter
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

k_B = 8.617e-5  # eV/K
hbar = 6.582e-16  # eV·s

print("✓ Imports complete")

In [ ]:
from extract_theta_from_optical import (
    MaterialParams,
    MATERIALS,
    generate_dummy_data,
    compute_memory_function,
    spectral_measure,
    compute_entropy,
    compute_energy,
    extract_theta,
    find_theta_critical,
    classify_material,
    extract_theta_full_pipeline
)
print("✓ Extraction module loaded")

## 1. Material Configuration

In [ ]:
MATERIAL_KEY = 'LSCO_x015'
mat = MATERIALS[MATERIAL_KEY]
T_array = np.array([100, 150, 200, 250, 300, 350])
print(f"Material: {mat.name}")
print(f"Structure: {mat.structure}")
print(f"T_c = {mat.Tc:.1f} K")
print(f"ω_p = {mat.omega_p:.2f} eV")
print(f"\nExpected Θ_c/T_c: {mat.expected_ratio:.3f} ± {mat.ratio_tolerance:.3f}")
print(f"\nTemperature points: {len(T_array)}")
print(f"Range: {T_array[0]:.0f} - {T_array[-1]:.0f} K")

## 2. Generate/Load Optical Conductivity Data

In [ ]:
T_demo = 200
omega, sigma_real, sigma_imag = generate_dummy_data(MATERIAL_KEY, T_demo, omega_max=5.0, n_points=500)
print(f"Generated σ(ω) at T = {T_demo} K")
print(f"Energy range: {omega[0]:.3f} - {omega[-1]:.3f} eV")
print(f"Number of points: {len(omega)}")
print(f"\nσ₁(0) = {sigma_real[0]:.2e}")
print(f"Peak σ₁ = {np.max(sigma_real):.2e} at ω = {omega[np.argmax(sigma_real)]:.3f} eV")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
axes[0].plot(omega, sigma_real, linewidth=2, label='σ₁(ω) [Real]')
axes[0].axvline(omega[np.argmax(sigma_real)], linestyle='--', alpha=0.5, label='Peak')
axes[0].set_ylabel('σ₁(ω) [arb. units]')
axes[0].set_title(f'Optical Conductivity: {mat.name} at T = {T_demo} K')
axes[0].legend()
axes[0].set_xlim(0, 5)
axes[1].plot(omega, sigma_imag, linewidth=2, label='σ₂(ω) [Imaginary]')
axes[1].axhline(0, linestyle='-', alpha=0.3)
axes[1].set_xlabel('ω [eV]')
axes[1].set_ylabel('σ₂(ω) [arb. units]')
axes[1].legend()
axes[1].set_xlim(0, 5)
plt.tight_layout()
plt.savefig('01_raw_optical_conductivity.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Raw data plotted")

## 3. Extract Memory Function M(ω)

In [ ]:
M_real, M_imag, M_diagnostics = compute_memory_function(omega, sigma_real, sigma_imag, mat.omega_p)
print("Memory Function M(ω) Diagnostics:")
print("="*50)
print(f"Causality OK: {M_diagnostics['causality_ok']}")
print(f"KK Consistency: {M_diagnostics.get('kk_ok', 'N/A')}")
if 'kk_violation' in M_diagnostics and not np.isnan(M_diagnostics['kk_violation']):
    print(f"KK Violation: {M_diagnostics['kk_violation']:.1%}")
    gate_status = "✓ PASS" if M_diagnostics['kk_ok'] else "✗ FAIL"
    print(f"KK Gate (<20%): {gate_status}")
print(f"Smoothness: {M_diagnostics['smoothness_max_abs_d2M1']:.2e}")
print("="*50)

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10))
axes[0].plot(omega, M_real, linewidth=2, label='Re M(ω)')
axes[0].axhline(0, linestyle='-', alpha=0.3)
axes[0].set_ylabel('Re M(ω) [eV]')
axes[0].set_title('Memory Function: Energy Component')
axes[0].legend()
axes[0].set_xlim(0, 5)
axes[1].plot(omega, M_imag, linewidth=2, label='Im M(ω)')
axes[1].axhline(0, linestyle='-', alpha=0.3)
axes[1].set_ylabel('Im M(ω) [eV]')
axes[1].set_title('Memory Function: Information Component')
axes[1].legend()
axes[1].set_xlim(0, 5)
ratio_M = np.where(np.abs(M_imag) > 1e-6, M_real / M_imag, np.nan)
axes[2].plot(omega, ratio_M, linewidth=2, label='Re M / Im M')
axes[2].axhline(1.45, linestyle='--', alpha=0.7, label='Expected (standard): 1.45')
axes[2].axhline(0.95, linestyle='--', alpha=0.7, label='Expected (infinite): 0.95')
axes[2].set_xlabel('ω [eV]')
axes[2].set_ylabel('Re M / Im M')
axes[2].set_title('Ratio: Energy / Information (Structural Fingerprint)')
axes[2].legend()
axes[2].set_xlim(0, 5)
axes[2].set_ylim(-2, 4)
plt.tight_layout()
plt.savefig('02_memory_function.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Memory function plotted")

## 4. Construct Spectral Measure π(ω)

In [ ]:
pi = spectral_measure(omega, sigma_real, method='f-sum')
norm_check = integrate.trapz(pi, omega)
print(f"Spectral Measure π(ω):")
print("="*50)
print(f"Normalization: ∫π(ω)dω = {norm_check:.6f}")
print(f"Norm gate (=1): {'✓ PASS' if np.abs(norm_check - 1.0) < 1e-3 else '✗ FAIL'}")
print(f"Max π(ω): {np.max(pi):.4f}")
print(f"Peak at: ω = {omega[np.argmax(pi)]:.3f} eV")
print(f"Support: [{omega[pi > 0.001*np.max(pi)][0]:.3f}, {omega[pi > 0.001*np.max(pi)][-1]:.3f}] eV")
print("="*50)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.plot(omega, pi, linewidth=2.5, label='π(ω) = σ₁/∫σ₁')
ax.fill_between(omega, 0, pi, alpha=0.3)
ax.axvline(omega[np.argmax(pi)], linestyle='--', alpha=0.5, label=f'Peak: {omega[np.argmax(pi)]:.2f} eV')
ax.set_xlabel('ω [eV]')
ax.set_ylabel('π(ω) [normalized]')
ax.set_title('Spectral Information Measure')
ax.legend()
ax.set_xlim(0, 5)
plt.tight_layout()
plt.savefig('03_spectral_measure.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Spectral measure plotted")

## 5. Compute Energy E[π] and Entropy S[π]

In [ ]:
E_canonical = compute_energy(omega, pi, M_real, method='canonical')
E_memory = compute_energy(omega, pi, M_real, method='memory')
E_corrected = compute_energy(omega, pi, M_real, method='corrected')
S, S_diag = compute_entropy(omega, pi)
print(f"Energy & Entropy:")
print("="*50)
print(f"E[π] (canonical, ℏω):  {E_canonical:.4f} eV")
print(f"E[π] (memory, Re M):   {E_memory:.4f} eV")
print(f"E[π] (corrected):      {E_corrected:.4f} eV")
print(f"\nS[π] (Shannon):        {S:.4f}")
print(f"S per mode:             {S_diag['S_per_mode']:.6f}")
print("="*50)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
epsilon_canonical = hbar * omega
epsilon_memory = M_real
W = np.max(omega) * 0.7
epsilon_corrected = hbar * omega * (1 + 0.5 * (omega / W)**2)
axes[0,0].plot(omega, epsilon_canonical, linewidth=2, label='ε = ℏω (canonical)')
axes[0,0].plot(omega, epsilon_memory, linewidth=2, label='ε = Re M(ω)')
axes[0,0].plot(omega, epsilon_corrected, linewidth=2, label='ε = ℏω[1+α(ω/W)²]')
axes[0,0].set_xlabel('ω [eV]')
axes[0,0].set_ylabel('ε(ω) [eV]')
axes[0,0].set_title('Energy Kernels')
axes[0,0].legend()
axes[0,0].set_xlim(0, 5)
axes[0,1].plot(omega, epsilon_canonical * pi, linewidth=2, label='Canonical')
axes[0,1].plot(omega, epsilon_memory * pi, linewidth=2, label='Memory')
axes[0,1].plot(omega, epsilon_corrected * pi, linewidth=2, label='Corrected')
axes[0,1].fill_between(omega, 0, epsilon_canonical * pi, alpha=0.2)
axes[0,1].set_xlabel('ω [eV]')
axes[0,1].set_ylabel('ε(ω) · π(ω)')
axes[0,1].set_title('Energy Integrand')
axes[0,1].legend()
axes[0,1].set_xlim(0, 5)
pi_safe = np.where(pi > 1e-12, pi, 1e-12)
entropy_integrand = -pi * np.log(pi_safe)
axes[1,0].plot(omega, entropy_integrand, linewidth=2)
axes[1,0].fill_between(omega, 0, entropy_integrand, alpha=0.3)
axes[1,0].set_xlabel('ω [eV]')
axes[1,0].set_ylabel('-π(ω) ln π(ω)')
axes[1,0].set_title('Entropy Integrand')
axes[1,0].set_xlim(0, 5)
axes[1,1].axis('off')
summary_text = f"""
ENERGY & ENTROPY SUMMARY
{'='*35}

E (canonical):  {E_canonical:.4f} eV
E (memory):     {E_memory:.4f} eV
E (corrected):  {E_corrected:.4f} eV

Variation:      {100*np.std([E_canonical, E_memory, E_corrected])/np.mean([E_canonical, E_memory, E_corrected]):.1f}%

S (Shannon):    {S:.4f}
S/mode:         {S_diag['S_per_mode']:.6f}

Peak entropy contribution:
  at ω = {S_diag['peak_omega']:.3f} eV
"""
axes[1,1].text(0.1, 0.9, summary_text, transform=axes[1,1].transAxes,
              fontsize=11, verticalalignment='top', family='monospace')
plt.tight_layout()
plt.savefig('04_energy_entropy.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Energy and entropy analyzed")

## 6. Extract Θ(T) with Sensitivity Analysis

In [ ]:
Theta_canonical, _ = extract_theta(omega, pi, M_real, energy_method='canonical')
Theta_memory, _ = extract_theta(omega, pi, M_real, energy_method='memory')
Theta_corrected, _ = extract_theta(omega, pi, M_real, energy_method='corrected')
Theta_mean = np.mean([Theta_canonical, Theta_memory, Theta_corrected])
Theta_std = np.std([Theta_canonical, Theta_memory, Theta_corrected])
sensitivity = 100 * Theta_std / Theta_mean
print(f"\nΘ Extraction (Single Temperature):")
print("="*50)
print(f"T = {T_demo} K")
print(f"\nΘ (canonical):  {Theta_canonical:.4f} eV")
print(f"Θ (memory):     {Theta_memory:.4f} eV")
print(f"Θ (corrected):  {Theta_corrected:.4f} eV")
print(f"\nMean:           {Theta_mean:.4f} eV")
print(f"Std:            {Theta_std:.4f} eV")
print(f"Sensitivity:    {sensitivity:.1f}%")
print(f"\nSensitivity gate (<10%): {'✓ PASS' if sensitivity < 10 else '✗ FAIL'}")
print("="*50)

## 7. Full Temperature Series: Θ(T) Evolution

In [ ]:
print("Running full temperature series...\n")
results = extract_theta_full_pipeline(MATERIAL_KEY, T_array, use_dummy=True, energy_method='canonical', verbose=False)
print("\n✓ Full pipeline complete")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
T_plot = results['T_array']
Theta_plot = results['Theta_array']
Tc = results['Tc']
Theta_c = results['Theta_c']
axes[0,0].plot(T_plot, Theta_plot, 'o-', linewidth=2, markersize=8, label='Θ(T)')
axes[0,0].axvline(Tc, linestyle='--', linewidth=2, alpha=0.7, label=f'T_c = {Tc:.0f} K')
axes[0,0].axhline(Theta_c, linestyle='--', linewidth=2, alpha=0.7, label=f'Θ_c = {Theta_c:.3f} eV')
axes[0,0].plot(Tc, Theta_c, '*', markersize=20, label='Critical Point')
axes[0,0].set_xlabel('Temperature [K]')
axes[0,0].set_ylabel('Θ(T) [eV]')
axes[0,0].set_title('Information Temperature vs Temperature')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)
ratio_T = Theta_plot / T_plot
axes[0,1].plot(T_plot, ratio_T, 's-', linewidth=2, markersize=8, label='Θ(T)/T')
axes[0,1].axvline(Tc, linestyle='--', linewidth=2, alpha=0.7)
axes[0,1].axhline(results['ratio'], linestyle='--', linewidth=2, alpha=0.7, label=f"Θ_c/T_c = {results['ratio']:.3f}")
axes[0,1].set_xlabel('Temperature [K]')
axes[0,1].set_ylabel('Θ(T)/T [eV/K]')
axes[0,1].set_title('Normalized Information Temperature')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)
dTheta_dT = np.gradient(Theta_plot, T_plot)
axes[1,0].plot(T_plot, dTheta_dT, '^-', linewidth=2, markersize=8, label='dΘ/dT')
axes[1,0].axvline(Tc, linestyle='--', linewidth=2, alpha=0.7)
axes[1,0].axhline(0, linestyle='-', alpha=0.3)
idx_max_deriv = np.argmax(np.abs(dTheta_dT))
axes[1,0].plot(T_plot[idx_max_deriv], dTheta_dT[idx_max_deriv], '*', markersize=15, label=f'Max at T={T_plot[idx_max_deriv]:.0f}K')
axes[1,0].set_xlabel('Temperature [K]')
axes[1,0].set_ylabel('dΘ/dT [eV/K]')
axes[1,0].set_title('Temperature Derivative (Contrast Method)')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)
methods_data = results['Theta_c_diagnostics']['methods']
method_names = list(methods_data.keys())
method_values = [methods_data[m][0] for m in method_names]
x_pos = np.arange(len(method_names))
axes[1,1].barh(x_pos, method_values, alpha=0.7)
axes[1,1].axvline(Theta_c, linestyle='--', linewidth=2, label=f'Consensus: {Theta_c:.3f} eV')
axes[1,1].set_yticks(x_pos)
axes[1,1].set_yticklabels(method_names)
axes[1,1].set_xlabel('Θ_c [eV]')
axes[1,1].set_title('Critical Point: Method Comparison')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3, axis='x')
agreement = results['Theta_c_diagnostics']['agreement']
axes[1,1].text(0.95, 0.05, f'Agreement: {100*(1-agreement):.1f}%', transform=axes[1,1].transAxes, fontsize=10, verticalalignment='bottom', horizontalalignment='right')
plt.tight_layout()
plt.savefig('05_theta_temperature_series.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Temperature series plotted")

## 8. Classification & Validation

In [ ]:
class_name = results['classification']
confidence = results['confidence']
ratio_measured = results['ratio']
ratio_expected = mat.expected_ratio
tolerance = mat.ratio_tolerance
validation = results['validation']
print("\n" + "="*60)
print("CLASSIFICATION & VALIDATION REPORT")
print("="*60)
print(f"\nMaterial: {mat.name}")
print(f"Structure: {mat.structure}")
print(f"\nMEASUREMENTS:")
print(f"  T_c = {results['Tc']:.1f} K")
print(f"  Θ_c = {results['Theta_c']:.4f} eV")
print(f"  Θ_c/T_c = {ratio_measured:.4f}")
print(f"\nEXPECTED (from theory):")
print(f"  Θ_c/T_c = {ratio_expected:.3f} ± {tolerance:.3f}")
print(f"\nCLASSIFICATION:")
print(f"  Class: {class_name.upper()}")
print(f"  Confidence: {confidence:.1%}")
print(f"\nVALIDATION:")
print(f"  Error: {validation['relative_error']:.1%}")
print(f"  Within 2σ: {validation['within_tolerance']}")
print(f"  Status: {validation['status']}")
print("\n" + "="*60)
agreement_pct = 100 * (1 - results['Theta_c_diagnostics']['agreement'])
methods_data = results['Theta_c_diagnostics']['methods']
for method, (val, T_val) in methods_data.items():
    print(f"  {method:15s}: Θ_c = {val:.4f} eV (at T = {T_val:.0f} K)")
print(f"  Consensus: {results['Theta_c']:.4f} eV")
print(f"  Agreement: {agreement_pct:.1f}%")
print(f"  Gate (<10%): {'✓ PASS' if agreement_pct > 90 else '✗ FAIL'}")
print("="*60 + "\n")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
ax = axes[0]
classes = ['Standard', 'Infinite-Layer']
expected_ratios = [1.45, 0.95]
tolerances = [0.12, 0.05]
for i, (cls, ratio, tol) in enumerate(zip(classes, expected_ratios, tolerances)):
    ax.axhspan(ratio - 2*tol, ratio + 2*tol, alpha=0.2, label=f'{cls} (2σ)')
    ax.axhline(ratio, linestyle='--', linewidth=2)
ax.plot(1, ratio_measured, '*', markersize=25, label=f'{mat.name}', zorder=10)
ax.axhline(ratio_measured, linestyle=':', alpha=0.5)
ax.set_xlim(0.5, 1.5)
ax.set_ylim(0.7, 1.8)
ax.set_xticks([])
ax.set_ylabel('Θ_c/T_c')
ax.set_title('Structural Classification')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
ax = axes[1]
metrics = ['Expected', 'Measured', 'Lower 2σ', 'Upper 2σ']
values = [ratio_expected, ratio_measured, ratio_expected - 2*tolerance, ratio_expected + 2*tolerance]
bars = ax.barh(metrics, values, alpha=0.7, edgecolor='black')
for bar, val in zip(bars, values):
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2, f' {val:.3f}', ha='left', va='center', fontsize=10, fontweight='bold')
ax.set_xlabel('Θ_c/T_c')
ax.set_title('Validation: Expected vs Measured')
ax.grid(True, alpha=0.3, axis='x')
status_color = 'green' if validation['status'] == 'PASS' else 'red'
ax.text(0.95, 0.95, f"Status: {validation['status']}", transform=ax.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')
plt.tight_layout()
plt.savefig('06_classification_validation.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Classification plotted")

## 9. Falsification Dashboard

In [ ]:
gates = {
    'KK Consistency': {
        'value': M_diagnostics.get('kk_violation', 0),
        'threshold': 0.20,
        'unit': '',
        'passed': M_diagnostics.get('kk_ok', True)
    },
    'Normalization': {
        'value': np.abs(norm_check - 1.0),
        'threshold': 0.001,
        'unit': '',
        'passed': np.abs(norm_check - 1.0) < 0.001
    },
    'ε Sensitivity': {
        'value': sensitivity / 100,
        'threshold': 0.10,
        'unit': '',
        'passed': sensitivity < 10
    },
    'Method Agreement': {
        'value': results['Theta_c_diagnostics']['agreement'],
        'threshold': 0.10,
        'unit': '',
        'passed': results['Theta_c_diagnostics']['agreement'] < 0.10
    },
    'Classification': {
        'value': validation['relative_error'],
        'threshold': 2 * tolerance / ratio_expected,
        'unit': '',
        'passed': validation['within_tolerance']
    }
}
print("\n" + "="*70)
print("FALSIFICATION GATE DASHBOARD")
print("="*70)
print(f"{'Gate':<20} {'Value':<12} {'Threshold':<12} {'Status':<10}")
print("-"*70)
all_passed = True
for gate_name, gate_data in gates.items():
    val = gate_data['value']
    thresh = gate_data['threshold']
    passed = gate_data['passed']
    status = "✓ PASS" if passed else "✗ FAIL"
    print(f"{gate_name:<20} {val:<12.4f} {thresh:<12.4f} {status:<10}")
    if not passed:
        all_passed = False
print("="*70)
overall_status = "✓ ALL GATES PASSED" if all_passed else "✗ SOME GATES FAILED"
print(f"\nOVERALL: {overall_status}")
print("="*70 + "\n")

In [ ]:
fig = plt.figure(figsize=(14, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
ax1 = fig.add_subplot(gs[0, :])
gate_names = list(gates.keys())
gate_values = [g['value'] / g['threshold'] for g in gates.values()]
gate_passed = [g['passed'] for g in gates.values()]
bars = ax1.barh(gate_names, gate_values, alpha=0.6, edgecolor='black')
ax1.axvline(1.0, linestyle='--', linewidth=2, label='Threshold')
ax1.set_xlabel('Value / Threshold')
ax1.set_title('Falsification Gates Status')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='x')
for i, (bar, passed) in enumerate(zip(bars, gate_passed)):
    label = '✓' if passed else '✗'
    ax1.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height()/2, label, ha='left', va='center', fontsize=14, fontweight='bold')
ax2 = fig.add_subplot(gs[1, :])
ax2.plot(T_plot, Theta_plot, 'o-', linewidth=2, markersize=6, label='Θ(T)')
ax2.axvline(Tc, linestyle='--', alpha=0.7, label=f'T_c = {Tc:.0f} K')
ax2.plot(Tc, Theta_c, '*', markersize=15)
ax2.set_xlabel('T [K]')
ax2.set_ylabel('Θ(T) [eV]')
ax2.set_title('Information Temperature')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax3 = fig.add_subplot(gs[2, 0])
confidence_vals = [confidence, 1-confidence]
confidence_labels = [f'{class_name}\n{confidence:.1%}', f'Other\n{1-confidence:.1%}']
ax3.pie(confidence_vals, labels=confidence_labels, autopct='', startangle=90)
ax3.set_title('Classification\nConfidence')
ax4 = fig.add_subplot(gs[2, 1])
ax4.axis('off')
val_status = validation['status']
ax4.text(0.5, 0.5, val_status, ha='center', va='center', fontsize=16, fontweight='bold')
ax4.set_xlim(0, 1)
ax4.set_ylim(0, 1)
ax4.set_title('Validation\nStatus')
ax5 = fig.add_subplot(gs[2, 2])
ax5.axis('off')
summary_stats = f"""
SUMMARY STATISTICS
{'='*25}

Θ_c/T_c:  {ratio_measured:.4f}
Expected: {ratio_expected:.3f}
Error:    {validation['relative_error']:.1%}

Methods:
  Agreement: {100*(1-results['Theta_c_diagnostics']['agreement']):.1f}%
  Consensus: {Theta_c:.3f} eV

Gates:
  Passed: {sum(gate_passed)}/{len(gates)}
"""
ax5.text(0.1, 0.9, summary_stats, transform=ax5.transAxes, fontsize=10, verticalalignment='top', family='monospace')
plt.savefig('07_falsification_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Dashboard complete")

## 10. Export Results

In [ ]:
results_df = pd.DataFrame({
    'Material': [mat.name],
    'Structure': [mat.structure],
    'Tc_K': [results['Tc']],
    'Theta_c_eV': [results['Theta_c']],
    'Ratio_measured': [ratio_measured],
    'Ratio_expected': [ratio_expected],
    'Error_relative': [validation['relative_error']],
    'Classification': [class_name],
    'Confidence': [confidence],
    'Validation_status': [validation['status']],
    'Gates_passed': [sum([g['passed'] for g in gates.values()])],
    'Gates_total': [len(gates)],
})
results_df.to_csv('theta_extraction_results.csv', index=False)
print("✓ Results exported to theta_extraction_results.csv")
theta_series_df = pd.DataFrame({'T_K': T_plot,'Theta_eV': Theta_plot,'Theta_over_T': Theta_plot / T_plot})
theta_series_df.to_csv('theta_temperature_series.csv', index=False)
print("✓ Θ(T) series exported to theta_temperature_series.csv")
results_df

## 11. Next Steps & Recommendations

In [ ]:
recommendations = f"""
{'='*70}
NEXT STEPS & RECOMMENDATIONS
{'='*70}

Current Status: {validation['status']}
Gates Passed: {sum([g['passed'] for g in gates.values()])}/{len(gates)}

IMMEDIATE ACTIONS:
1) Replace dummy data with real σ(ω,T) at: 0.5Tc, 0.7Tc, 0.9Tc, 1.1Tc, 1.5Tc, 2Tc
2) Validate on LSCO, YBCO, Bi2212 (standard) and CaCuO2 (infinite-layer)
3) Sensitivity sweeps: ε(ω), cutoff Ω, T-resolution
4) Publish Methods → then Theory if universality holds

FALSIFICATION: If any gate fails, revise step-by-step (M extraction, Θ_c definition, class priors)
"""
print(recommendations)
with open('next_steps_recommendations.txt', 'w') as f:
    f.write(recommendations)
print("\n✓ Recommendations saved to next_steps_recommendations.txt")

## 12. Generate Report

In [ ]:
from datetime import datetime
methods_data = results['Theta_c_diagnostics']['methods']
agreement_pct = 100*(1-results['Theta_c_diagnostics']['agreement'])
report = f"""
{'='*70}
Θ-MECHANISM EXTRACTION REPORT
{'='*70}
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Material: {mat.name}
Structure: {mat.structure}
MEASUREMENTS
T_c = {results['Tc']:.1f} K
Θ_c = {results['Theta_c']:.4f} eV
Θ_c/T_c = {results['ratio']:.4f}
THEORY COMPARISON
Expected Θ_c/T_c = {ratio_expected:.3f} ± {tolerance:.3f}
Relative Error: {validation['relative_error']:.1%}
Within 2σ: {validation['within_tolerance']}
CLASSIFICATION: {class_name.upper()} ({confidence:.1%})
CRITICAL METHODS:
"""
for method, (val, T_val) in methods_data.items():
    report += f"{method:18s}: Θ_c = {val:.4f} eV (T = {T_val:.0f} K)\n"
report += f"""Consensus: Θ_c = {results['Theta_c']:.4f} eV
Agreement: {agreement_pct:.1f}%
FALSIFICATION GATES:
"""
for gate_name, gate_data in gates.items():
    status = "PASS" if gate_data['passed'] else "FAIL"
    report += f"{gate_name:20s}: {status}\n"
overall_status = "ALL GATES PASSED" if all([g['passed'] for g in gates.values()]) else "SOME GATES FAILED"
report += f"Overall: {overall_status}\n"
with open('theta_extraction_report.txt', 'w') as f:
    f.write(report)
print(report)
print("\n✓ Full report saved to theta_extraction_report.txt")
print("\nGenerated files:\n  - 01_raw_optical_conductivity.png\n  - 02_memory_function.png\n  - 03_spectral_measure.png\n  - 04_energy_entropy.png\n  - 05_theta_temperature_series.png\n  - 06_classification_validation.png\n  - 07_falsification_dashboard.png\n  - theta_extraction_results.csv\n  - theta_temperature_series.csv\n  - theta_extraction_report.txt\n  - next_steps_recommendations.txt")